In [ ]:
from typing import List
from typing import Tuple
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

from tqdm import tqdm

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'grid.linestyle':'--'})

# Linear regression

Linear regression is a statistical method and a fundamental type of predictive analytical model used to describe the relationship between a dependent variable and one or more independent variables. In this class, we will use the "Auto MPG" dataset to learn and practice linear regression.

## The data

The "Auto MPG" dataset is one of the classic datasets available in the UCI Machine Learning Repository. It contains city-cycle fuel consumption estimates for various automobiles produced in the 1970s and 1980s, making it a commonly used dataset for regression analysis tasks in machine learning.

### Data Set Information:

The data concerns city-cycle fuel consumption in miles per gallon (MPG) to be predicted in terms of 3 multivalued discrete and 5 continuous attributes.

* mpg: continuous
* cylinders: multi-valued discrete
* displacement: continuous
* horsepower: continuous
* weight: continuous
* acceleration: continuous
* model year: multi-valued discrete
* origin: multi-valued discrete
* car name: string (unique for each instance)

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/changyaochen/MECE4520/master/data/auto_mpg.csv")
data.head()

## Exploratory Data Analysis (EDA)

Exploratory Data Analysis (EDA) refers to the process of visually and analytically inspecting data to uncover its main characteristics, often using statistical graphics, plots, and information tables. EDA is essential in data science, and is a fundamental step before proceeding to more advanced machine learning or statistical modeling, as it helps analysts and researchers understand the nature of the data, its structure, anomalies, patterns, and relationships.

EDA is often an iterative process: as you discover one feature or pattern in your data, it might lead you to look for other related features or patterns. Below, we will use the "Auto MPG" dataset as an example.

In [ ]:
# distribution of the dependent variable
sns.histplot(x="mpg", data=data)
plt.tight_layout()

In [ ]:
# correlations
continuous_variables = [
    "displacement",
    "horsepower",
    "weight",
    "acceleration",
]

for variable in continuous_variables:
    plt.figure()
    sns.scatterplot(x=variable, y="mpg", data=data)
    plt.tight_layout()

## Simple linear regression

Simple linear regression refers to the special case of linear regression where there is only a single independent variable. Namely, it takes the form of:
$$y = \beta_0 + \beta_1 x,$$
where $y$ is the dependent variable, and $x$ is the single independent variable. The goal of the linear regression is to find the best values for the intercept $\beta_0$, and the slope $\beta_1$.

Below we will use the "Auto MPG" dataset as an example.

In [ ]:
y = data["mpg"]
x = data["weight"]

def simple_linear_regression(
    x: Union[List, np.ndarray, pd.Series], 
    y: Union[List, np.ndarray, pd.Series]) -> Tuple[float, float]:
    """Return the intercept and slope of a simple linear regression."""
    beta_1 = np.cov(x, y)[0][1] / np.cov(x, x)[0][1]
    beta_0 = np.mean(y) - beta_1 * np.mean(x)
    
    return beta_0, beta_1

beta_0, beta_1 = simple_linear_regression(x=x, y=y)

# calculate R^2
y_pred = beta_0 + beta_1 * x
SST = np.sum(np.square(y - np.mean(y)))
residual = y - y_pred
SSE = np.sum(np.square(residual))
r2 = 1 - SSE / SST

print(f"beta_0 is: {beta_0:5.4f}")
print(f"beta_1 is: {beta_1:5.4f}")
print(f"R-square is: {r2:5.4f}")

plt.figure()
x_range = np.linspace(start=np.min(x), stop=np.max(x), num=100)
sns.scatterplot(x="weight", y="mpg", data=data)
sns.lineplot(x=x_range, y=(beta_0 + beta_1 * x_range), color="red")
plt.tight_layout()

Equally important to, if not more important than, the model fitting, is the model/error analysis. Here we will demonstrate a simple residual analysis, to examine if its distirubtion agrees with our (normality) assumption> 

In [ ]:
# residual analysis
plt.figure()
sns.histplot(residual)
plt.tight_layout()
plt.show()

An important aspect in linear regression is that, the fitted parameters (_e.g._, intercept, slope), are just point estimates of the population parameter. Here the "population" refers to the underlying stochastic process that generates the observed data. 

Below, we will run a simulation: in each trial, we only sample 20% from the full observation, and run the simple linear regression. We will repeat the trial 100 times. This is to show that, the fitting parameters is a random variable: it depends on the exact data that is used. Therefore, as a random variable, we need to pay attention to its bias and variance.

In [ ]:
# accuracy of the coefficients
np.random.seed(42)
n_trials = 100

beta_0s, beta_1s = [], []
for _ in tqdm(range(n_trials)):
    sampling_proba = 0.2
    mask = np.random.choice([True, False], size=len(x), p=[sampling_proba, 1 - sampling_proba])
    x_sampled, y_sampled = x[mask], y[mask]
    beta_0, beta_1 = simple_linear_regression(x=x_sampled, y=y_sampled)
    beta_0s.append(beta_0)
    beta_1s.append(beta_1)

# plot the histograms
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 5))
sns.histplot(beta_0s, ax=axes[0])
sns.histplot(beta_1s, ax=axes[1])
plt.tight_layout()

# plot the fited lines
plt.figure()
x_range = np.linspace(start=np.min(x), stop=np.max(x), num=100)
for i in tqdm(range(len(beta_0s))):
    sns.lineplot(x=x_range, y=(beta_0s[i] + beta_1s[i] * x_range), color="red", alpha=0.1)
sns.scatterplot(x="weight", y="mpg", data=data)
plt.tight_layout()

In [ ]:
# confidence intervals
SE_beta_0 = (np.var(residual, ddof=2) * (1. / len(x) + (np.mean(x))**2 / np.sum((x - np.mean(x))**2)))**0.5
SE_beta_1 = (np.var(residual) / np.sum((x - np.mean(x))**2))**0.5 

print(f"The standard error for beta_0 is: {SE_beta_0:5.4f}")
print(f"The standard error for beta_1 is: {SE_beta_1:5.4f}")

In [ ]:
# simple linear regression with the `statsmodels` library
model_1 = smf.ols(formula='mpg ~ weight', data=data)
result_1 = model_1.fit()
print(result_1.summary())

In [ ]:
# linear regress in matrix format
X = np.hstack(
    (np.ones(shape=(len(x), 1)), 
     x.to_numpy().reshape(-1, 1)))

# point estimate
beta_matrix = np.linalg.inv(X.T @ X) @ X.T @ y
print("The estimates for beta are:")
print(beta_matrix)

# variance
se_matrix = np.sqrt(np.var(residual, ddof=2) * np.linalg.inv(X.T @ X))
print("\nThe standard error for beta are:")
print(se_matrix)

## Multi-variant linear regression

Multi-variant linear regression is a linear regression that takes more than one independent variable. 

In the following, we will use the Python `statsmodels` library to perform multi-variant linear regression.

In [ ]:
model_2 = smf.ols(formula='mpg ~ weight + displacement + horsepower + acceleration', data=data)
result_2 = model_2.fit()
print(result_2.summary())

In [ ]:
# correlation between the continuous variables
data[continuous_variables].corr()

## One-hot encoding

If the independent variable is of categorical type, we can not directly use them in regression. One-hot encoding is the solution to address this issue. 

At its core, one-hot encoding is about converting nominal categorical data into a format that can be used by machine learning algorithms. Categorical data refers to variables that can be divided into multiple categories but have no order or priority. These categories can be text labels, which many ML algorithms can't work with directly because they expect numerical input features.

With the `statsmodels` library, you simply wrap the independent variable with `C()`. If you are using other libraries, please refer to the corresponding APIs. For example, in the popular `scikit-learn` library, one can [use](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) the `OneHotEncoder` to achieve the same goal.

In [ ]:
# one-hot encode the categorical variables
model_3 = smf.ols(formula='mpg ~ weight + C(origin)', data=data)
result_3 = model_3.fit()
print(result_3.summary())